In [1]:
import numpy as np
from matplotlib import pyplot as plt

### Class Node
An object of this class represents one node in the finite element model. <br>

<b> Fields: </b> <br>
coords: list of floats, xyz coordinates of the node <br>
previous: Node object, corresponds to the previous node belonging to the same microtubule <br>
next: Node object, corresponds to the next node belonging to the same microtubule <br>
id: int, unique identifying number to reference this Node object <br>
connected: boolean, True if this node is connected to any other node <br>
mt: Microtubule object, refers to the microtubule to which this node belongs <br>

In [ ]:
class Node:
    def __init__(self, coords):
        self.coords = coords
        self.previous = None
        self.next = None
        self.id = None
        self.connected = False
        self.mt = None
    
    def set_id(self, index):
        self.id = index
        
    def set_mt(self, mt):
        self.mt = mt

### Class Microtubule
An object of this class represents one microtubule in the image stack. <br>

<b> Fields: </b> <br>

nodes: list of Node objects, holds all the nodes correponding to this microtubule <br>
vertices: list of Node objects, holds all of the vertices corresponding to this microtubule <br>
zstart: float, z coordinate of the first node in the microtubule <br>
zend: float, z coordinate of the last node in the microtubule <br>
numCL: int, number of crosslinks attached to this microtubule

In [ ]:
class Microtubule:
    def __init__(self):
        self.nodes = []
        self.vertices = []
        self.zstart = None
        self.zend = None
        self.numCL = 0
        
    def add_vertex(self, vertex):
        self.vertices.append(vertex)
        
    def add_node(self, node):
        self.nodes.append(node)
        
    def add_cl(self):
        self.numCL = self.numCL + 1

### Class CrossSection
An object of this class represents one cross section of the full axon model. <br>

<b> Fields: </b> <br>
z: float, z coordinate of this cross section <br>
mts: list of Microtubule objects, holds all microtubules that pass through this cross section <br>
nodes: list of Node objects, holds all nodes present in this cross section <br>
clPositions: list of tuples of Node objects, potential node pairs that could be connected by a crosslink <br>
<br>
<b> Methods: </b> <br>
computeCLloc: computes distances between pairs of nodes in this cross section then stores the pairs that are closer together than the specified threshold as potential crosslink locations

In [ ]:
class CrossSection:
    def __init__(self, z):
        self.z = z #double, z coordinate of this cross section
        self.mts = [] #List of Microtubule objects for microtubules crossing this section
        self.nodes = [] #List of Node objects for nodes found on this cross section
        self.clPositions = [] #List of tuples of Node objects that could be connected by a crosslink 
        
    def add_mt(self, mt):
        self.mts.append(mt)
        
    def add_node(self, node):
        self.nodes.append(node)
        
    def computeCLloc(self, thresh):
        for i, node1 in enumerate(self.nodes):
            for node2 in self.nodes[i+1:]:
                if (np.linalg.norm(node2.coords-node1.coords)<thresh):
                    if (node2.next != None):
                        self.clPositions.append((node1,node2.next))
                    if (node1.next != None):
                        self.clPositions.append((node2, node1.next))

## Main Code
The following code reads in the coordinate and connectivity information from the results of the image analysis and registration processes. It then randomly generates crosslinks between the microtubules and discretizes the microtubules and crosslinks into a finite element mesh. The final mesh information is output in the form of an ABAQUS input file.

In [ ]:
#reconstructing microtubules from the image analysis results

#read in coordinates and connectivity info from the registration results
vertices = []
with open("vertexCoords.txt", "r") as f:
    for line in f:
        vertices.append(np.fromstring(line[1:-2].strip(), sep = ' '))
        
connectivity = []
with open("connectivityTable.txt", "r") as f:
    for line in f:
        connectivity.append(eval(line))

#create and store Node objects for each microtubule coordinate
vertexObj = []

for coord in vertices:
    vertexObj.append(Node(coord))

#update connectivity information within Node objects by linking them together
for connection in connectivity:
    i = connection[0]
    j = connection[1]
    vertexObj[i].next = vertexObj[j]
    vertexObj[j].previous = vertexObj[i]
    vertexObj[i].connected = True
    vertexObj[j].connected = True

#create a Microtubule object for each microtubule in the axon, assign vertices to their corresponding Microtubules
mts = []

for vertex in vertexObj:
    if vertex.previous == None:
        mts.append(Microtubule())
        mts[-1].add_vertex(vertex)
        vertex.set_mt(mts[-1])
        
        current = vertex
        
        while current.next != None:
            mts[-1].add_vertex(current.next)
            current.next.set_mt(mts[-1])
            current = current.next

In [ ]:
#create finite element mesh for the microtubules

nElSlice = 1 #number of elements per slice
dz = 50. #nm - slice thickness
elSize = dz / nElSlice #element size

nSlice = 302 #number of TEM slices
nXS = (nSlice-1)*nElSlice + 1 #number of cross sections

#creating CrossSection objects for each cross section and assigning the corresponding Microtubules and Nodes
sections = []
idNum = 1

for i in range(nXS):
    sections.append(CrossSection(elSize*i))

for mt in mts:
    for vertex in mt.vertices:
        mt.nodes.append(vertex)
        vertex.set_id(idNum)
        idNum = idNum + 1
        
        zLoc = vertex.coords[2]
        index = int(round(zLoc/elSize))
        sections[index].add_mt(mt)
        sections[index].add_node(vertex)
        
        if vertex.previous != None:
            mt.nodes[-2].next = mt.nodes[-1]
            mt.nodes[-1].previous = mt.nodes[-2]
        
        if vertex.next != None:
            x1 = vertex.coords[0]
            x2 = vertex.next.coords[0]
            y1 = vertex.coords[1]
            y2 = vertex.next.coords[1]
            
            for i in range(nElSlice-1):
                x = x1 + (i+1)*((x2-x1)/nElSlice)
                y = y1 + (i+1)*((y2-y1)/nElSlice)
                z = vertex.coords[2] + (i+1)*elSize
                
                mt.nodes.append(Node(np.array([x,y,z])))
                mt.nodes[-1].set_id(idNum)
                mt.nodes[-1].set_mt(mt)
                idNum = idNum + 1
                
                index = int(round(z/elSize))
                sections[index].add_mt(mt)
                sections[index].add_node(mt.nodes[-1])
                
                mt.nodes[-2].next = mt.nodes[-1]
                mt.nodes[-1].previous = mt.nodes[-2]

#storing finite element mesh information for the microtubules
nodes_All = []
elements_MT = []

for mt in mts:
    for node in mt.nodes:
        nodes_All.append(node)
        if node.next != None:
            elements_MT.append((node.id, node.next.id))

In [ ]:
#generating crosslinks 

tauLen = 45. #nm

for section in sections:
    section.computeCLloc(tauLen)
    
clDensity = 10.0; #CL density numCL/um of MT length

elements_CL = []

for section in sections:
    
    clPerCS = elSize/1000 * len(section.mts)*clDensity; # number of crosslinks per cross section
    
    if section.clPositions:
        for i in range(int(clPerCS)):
            index = np.random.randint(0, len(section.clPositions))
            node1 = section.clPositions[index][0]
            node2 = section.clPositions[index][1]
            elements_CL.append((node1.id, node2.id))
            node1.mt.add_cl()
            node2.mt.add_cl()

In [ ]:
#write ABAQUS input file

# Abaqus units: length - nm, force - nN, time - s, stress - GPa

# First adjust node coordinates to be centered around zero
sumx = 0
sumy = 0
for node in nodes_All:
    sumx = sumx + node.coords[0]
    sumy = sumy + node.coords[1]

xavg = sumx/len(nodes_All)
yavg = sumy/len(nodes_All)

adjustment = [xavg, yavg, 0.0]

for node in nodes_All:
    node.coords = node.coords-adjustment

f = open("PLML.inp", "w+")
f.write("*HEADING \n")
f.write("three-dimensional reconstruction of Cueva PLML \n")
f.write("SI Units \n")
f.write("**\n** Model definition \n** \n")

f.write("*NODE, NSET=NSET_ALL\n") #node definitions

for node in nodes_All:
    f.write(str(node.id) + ', ' + str(tuple(node.coords))[1:-1]+'\n')

RPid = len(nodes_All)+1
f.write(str(RPid) + ", 0.0, 0.0, 15100.0 \n") #add a node for the reference point

f.write("*ELEMENT, TYPE=B33, ELSET=ELSET_MT \n") #microtubule elements

for i, element in enumerate(elements_MT):
    f.write(str(i+1) + ', ' + str(element)[1:-1]+'\n')

f.write("*ELEMENT, TYPE=T3D2, ELSET=ELSET_CL \n") #crosslink elements

elementNum = len(elements_MT) + 1

for i, element in enumerate(elements_CL):
    f.write(str(i+elementNum) + ', ' + str(element)[1:-1]+'\n')    
    
f.write("*NSET, NSET=NSET_REFPOINT\n")
        
f.write(str(RPid)+ '\n')

f.write("*NSET, NSET=NSET_FIXPOINTS\n")

numlines = len(fixNodes)//16 
if len(fixNodes) % 16 != 0:
    numlines = numlines + 1  
    
for i in range(numlines):
    for index, nodeID in enumerate(fixNodes[16*i:16*(i+1)]):
        if index != len(fixNodes[16*i:16*(i+1)]) - 1:
            f.write(str(nodeID) + ', ')
        else:
            f.write(str(nodeID) + '\n')

#create node set for nodes at fixed end of axon
fixedEndNodes = []
for node in sections[0].nodes:
    fixedEndNodes.append(node.id)

#figure out how many lines we need in the input file (max 16 nodes per line)
numlines = len(fixedEndNodes)//16 
if len(fixedEndNodes) % 16 != 0:
    numlines = numlines + 1  
    
f.write("*NSET, NSET=NSET_FIXEDEND\n")

for i in range(numlines):
    for index, node in enumerate(fixedEndNodes[16*i:16*(i+1)]):
        if index != len(fixedEndNodes[16*i:16*(i+1)]) - 1:
            f.write(str(node) + ', ')
        else:
            f.write(str(node) + '\n')

#create node set for nodes at displacement boundary condition
loadNodes = []
allOtherNodes = []

for node in nodes_All:
    if node.coords[2]==15050:
        loadNodes.append(node.id)
    else:
        allOtherNodes.append(node.id)

numlines = len(loadNodes)//16
if len(loadNodes) % 16 != 0:
    numlines = numlines + 1 

f.write("*NSET, NSET=NSET_LOADEND\n")

for i in range(numlines):
    for index, node in enumerate(loadNodes[16*i:16*(i+1)]):
        if index != len(loadNodes[16*i:16*(i+1)]) - 1:
            f.write(str(node) + ', ')
        else:
            f.write(str(node) + '\n')

numlines = len(allOtherNodes)//16
if len(allOtherNodes) % 16 != 0:
    numlines = numlines + 1
    
f.write("*NSET, NSET=NSET_ALLBUTLOAD\n")

for i in range(numlines):
    for index, node in enumerate(allOtherNodes[16*i:16*(i+1)]):
        if index != len(allOtherNodes[16*i:16*(i+1)]) - 1:
            f.write(str(node) + ', ')
        else:
            f.write(str(node) + '\n')
            
f.write("*SURFACE, TYPE=NODE, NAME=COUPLING_NODES\n")    
f.write("NSET_LOADEND\n")

f.write("*BEAM SECTION, ELSET=ELSET_MT, SECTION=CIRC, MATERIAL=MT \n")
f.write("11.28\n")
f.write("-1.0, 0.0, 0.0\n")

f.write("*SOLID SECTION, ELSET=ELSET_CL, MATERIAL=CL \n")
f.write("1.0\n")

#Material definitions
f.write("*MATERIAL, NAME=MT\n")
f.write("*ELASTIC \n")
f.write("1.2, 0.3\n")

f.write("*MATERIAL, NAME=CL\n")
f.write("*ELASTIC \n")
f.write("0.01, 0.3\n")

#coupling to reference point
f.write("*COUPLING, CONSTRAINT NAME=load_constraint, REF NODE=NSET_REFPOINT, SURFACE=COUPLING_NODES\n")
f.write("*KINEMATIC\n")
f.write("1, 3\n")

#fix nodes at one end
f.write("*BOUNDARY\n")
f.write("NSET_FIXEDEND, 3\n")

f.write("*BOUNDARY\n")
f.write("NSET_FIXPOINTS, 1,6\n")

f.write("*BOUNDARY\n")
f.write("NSET_REFPOINT, 1, 2\n")

#fix rotations
f.write("*BOUNDARY\n")
f.write("NSET_ALL, 4, 6\n")

#add load step
f.write("*STEP, NAME=LOADSTEP, NLGEOM=YES\n")
f.write("*STATIC\n")
f.write("0.01, 1, 10E-5\n")

f.write("*BOUNDARY, TYPE=DISPLACEMENT\n")
f.write("NSET_REFPOINT, 3, 3, 250.\n")

#output
f.write("*OUTPUT, FIELD\n")
f.write("*NODE OUTPUT\n")
f.write("RF, U\n")
f.write("*ELEMENT OUTPUT, DIRECTIONS=YES\n")
f.write("EE, ELEDEN, NE, S\n")
f.write("*OUTPUT, HISTORY\n")
f.write("*NODE OUTPUT, NSET=NSET_REFPOINT\n")
f.write("RF3\n")

f.write("*END STEP")

f.close()